In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import base
from src.agents import DeepQAgent

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Environment

In [3]:
env = gym.make('Breakout-v0')

In [4]:
env.observation_space

Box(210, 160, 3)

In [5]:
env.action_space

Discrete(4)

# Agent

In [12]:
agent = DeepQAgent(env, learning_rate=0.1, frames_per_action=2)
agent

DeepQAgent(
    env=<TimeLimit<AtariEnv<Breakout-v0>>>,
    learning_rate=0.1,
    discount_factor=0.99,
    exploration_rate=1.0,
    exploration_decay=0.9998,
    exploration_min=0.1,
    image_size=(84, 84),
    frames_per_action=2,
    replay_size=20000
)

In [10]:
agent.run()

array([-28., -38.,  -8.,  -8., -40., -28., -39.,  -9.,  -8.,  -8., -29.,
       -29., -30., -38., -10., -40., -29., -29., -37.,  -9., -17., -39.,
       -10., -29.,  -8., -30., -39., -18., -27., -20.])

In [ ]:
agent.train(episodes=30)

-10.0 nan
-10.0 nan
-17.0 nan
-10.0 nan
-20.0 nan
-10.0 nan


In [ ]:
agent.run()